# 2편: 완전연결 신경망으로 FashionMNIST 분류하기

이 노트북에서는 다음을 학습합니다:
- FashionMNIST 데이터셋의 구조와 시각화
- DataLoader를 사용한 배치 학습
- 1계층 → 다계층 완전연결(FC) 신경망 구현
- **파라미터 폭발 문제**를 직접 체감

## 1. FashionMNIST 데이터셋

FashionMNIST는 Zalando에서 공개한 패션 아이템 이미지 데이터셋입니다.
- **이미지**: 28 × 28 그레이스케일
- **학습 데이터**: 60,000장
- **테스트 데이터**: 10,000장
- **클래스**: 10가지 (T-shirt, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot)

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import time

# FashionMNIST 다운로드
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# 클래스 이름 매핑
class_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]

print(f"학습 데이터: {len(training_data)}장")
print(f"테스트 데이터: {len(test_data)}장")

In [ ]:
# 샘플 이미지 시각화
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    image, label = training_data[i]
    ax.imshow(image.squeeze(), cmap="gray")
    ax.set_title(class_names[label], fontsize=10)
    ax.axis("off")
plt.suptitle("FashionMNIST Samples", fontsize=14)
plt.tight_layout()
plt.show()

## 2. DataLoader로 배치 학습 준비

- **Dataset**: 개별 데이터(이미지, 레이블) 접근
- **DataLoader**: 배치 단위로 데이터를 묶어서 반복자 제공
- **batch_size**: 한 번에 학습할 데이터 수

In [ ]:
batch_size = 128
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

print(f"전체 학습 데이터: {len(training_data)}장")
print(f"배치 크기: {batch_size}")
print(f"배치 수: {len(train_dataloader)} (= {len(training_data)} / {batch_size} 올림)")

# 첫 번째 배치 확인
for images, labels in train_dataloader:
    print(f"\n배치 이미지 shape: {images.shape}")
    print(f"  → {images.shape[0]}장 × {images.shape[2]}×{images.shape[3]} 이미지 × {images.shape[1]} 채널")
    print(f"배치 레이블 shape: {labels.shape}")
    print(f"첫 번째 레이블: {labels[0]} ({class_names[labels[0]]})")
    print(f"\nFlatten 후: {images.view(-1, 28 * 28).shape}  (28×28=784 차원 벡터)")
    break

## 3. 1계층 완전연결 신경망

가장 단순한 구조: 784차원 입력 → 10개 클래스 출력

```
이미지 (28×28) → Flatten (784) → Linear(784, 10) → 출력 (10 클래스)
```

파라미터 수: 784 × 10 + 10(bias) = **7,850개**

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"사용 디바이스: {device}")

class SingleLayerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(28 * 28, 10)

    def forward(self, x):
        return self.linear(x)

single_net = SingleLayerNet().to(device)
print(single_net)

# 파라미터 수 계산
total_params = sum(p.numel() for p in single_net.parameters())
print(f"\n총 파라미터 수: {total_params:,}개")

In [ ]:
# 1계층 모델 학습
optimizer = torch.optim.SGD(single_net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss().to(device)

single_costs = []
single_times = []

for epoch in range(10):
    start_time = time.time()
    avg_cost = 0
    total_batch = len(train_dataloader)

    for X, Y in train_dataloader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = single_net(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost.item() / total_batch

    elapsed = time.time() - start_time
    single_costs.append(avg_cost)
    single_times.append(elapsed)
    print(f"Epoch {epoch+1:02d} | cost = {avg_cost:.6f} | time = {elapsed:.2f}s")

print(f"\n평균 에포크 시간: {sum(single_times)/len(single_times):.2f}s")

## 4. 다계층 완전연결 신경망

계층을 깊게 쌓으면 더 복잡한 패턴을 학습할 수 있지만, **파라미터 수가 폭발적으로 증가**합니다.

```
이미지(784) → Linear(784, 7840) → Linear(7840, 7840) → Linear(7840, 10) → 출력
```

파라미터 수:
- 1층: 784 × 7,840 + 7,840 = 6,154,240
- 2층: 7,840 × 7,840 + 7,840 = 61,473,440
- 3층: 7,840 × 10 + 10 = 78,410
- **합계: 약 6,770만개** (1계층 대비 약 8,600배!)

In [ ]:
class MultiLayerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(28 * 28, 28 * 28 * 10)
        self.linear2 = nn.Linear(28 * 28 * 10, 28 * 28 * 10)
        self.linear3 = nn.Linear(28 * 28 * 10, 10)

    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        return x

multi_net = MultiLayerNet().to(device)
print(multi_net)

total_params_multi = sum(p.numel() for p in multi_net.parameters())
print(f"\n총 파라미터 수: {total_params_multi:,}개")
print(f"1계층 대비 {total_params_multi / total_params:.0f}배 증가!")

In [ ]:
# 다계층 모델 학습
optimizer_multi = torch.optim.SGD(multi_net.parameters(), lr=0.001, momentum=0.9)

multi_costs = []
multi_times = []

for epoch in range(10):
    start_time = time.time()
    avg_cost = 0
    total_batch = len(train_dataloader)

    for X, Y in train_dataloader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer_multi.zero_grad()
        hypothesis = multi_net(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer_multi.step()

        avg_cost += cost.item() / total_batch

    elapsed = time.time() - start_time
    multi_costs.append(avg_cost)
    multi_times.append(elapsed)
    print(f"Epoch {epoch+1:02d} | cost = {avg_cost:.6f} | time = {elapsed:.2f}s")

print(f"\n평균 에포크 시간: {sum(multi_times)/len(multi_times):.2f}s")

## 5. 비교: 1계층 vs 다계층 FC

두 모델의 학습 과정을 비교하여 파라미터 폭발의 영향을 확인합니다.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Cost 비교
epochs = range(1, 11)
axes[0].plot(epochs, single_costs, "o-", label=f"1-Layer ({total_params:,} params)")
axes[0].plot(epochs, multi_costs, "s-", label=f"3-Layer ({total_params_multi:,} params)")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Cost")
axes[0].set_title("Cost Comparison")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 시간 비교
axes[1].plot(epochs, single_times, "o-", label="1-Layer")
axes[1].plot(epochs, multi_times, "s-", label="3-Layer")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Time (seconds)")
axes[1].set_title("Training Time per Epoch")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n=== 요약 ===")
print(f"{'':15s} {'1계층':>12s} {'3계층':>12s} {'배율':>8s}")
print(f"{'파라미터 수':15s} {total_params:>12,} {total_params_multi:>12,} {total_params_multi/total_params:>7.0f}x")
print(f"{'평균 시간(s)':15s} {sum(single_times)/10:>12.2f} {sum(multi_times)/10:>12.2f} {sum(multi_times)/sum(single_times):>7.1f}x")
print(f"{'최종 cost':15s} {single_costs[-1]:>12.6f} {multi_costs[-1]:>12.6f}")

## 완전연결 신경망의 한계

현재는 28×28의 아주 작은 그레이스케일 이미지를 학습하는 경우입니다.

하지만 실제에서는:
- **4K 이미지**: 3840 × 2160 × 3 = 24,883,200 차원
- **수십~수백 계층**으로 구성
- **수백만 장**의 학습 데이터

FC 레이어로 4K 이미지를 학습하면 **1계층만으로도 수조 개의 파라미터**가 필요합니다.
이는 메모리와 연산 측면에서 학습이 **사실상 불가능**합니다.

근본적인 문제: FC 레이어는 **이미지의 공간적 구조(인접 픽셀 간의 관계)**를 전혀 활용하지 못합니다.

→ **다음 노트북에서 CNN(합성곱 신경망)**으로 이 문제를 해결합니다.